In [ ]:
import tensorflow as tf 
import numpy as np 

tf.logging.set_verbosity(30)

In [ ]:
# -------------------------------------------------------------------------
#  => Cluster definition
# -------------------------------------------------------------------------

cluster = tf.train.ClusterSpec({
    "worker": [
        "test-tf-cluster-worker-0.marathon.l4lb.thisdcos.directory:2333",
        "test-tf-cluster-worker-1.marathon.l4lb.thisdcos.directory:2333",
        "test-tf-cluster-worker-2.marathon.l4lb.thisdcos.directory:2333"
    ],
    "ps":[
        "test-tf-cluster-ps-0.marathon.l4lb.thisdcos.directory:2333"
    ]
})

test_tf_cluster = cluster.as_dict()

In [ ]:
test_tf_cluster

In [ ]:
### Problema con los flags de Tensorflow en Notebook -> Tienes que reiniciar para poder volver a re-ejecutar el codigo

tf.app.flags.DEFINE_integer("numsamples", "1000", "Number of samples per server") 
FLAGS = tf.app.flags.FLAGS 

In [ ]:
print ("Sample number per server: " + str(FLAGS.numsamples) ) 

# This is the list containing the sumation of samples on any node 
c=[] 

def generate_sum(): 
    
    # Points generation (x1, x2)
    i = tf.constant( np.random.uniform(size=FLAGS.numsamples*2), shape=[FLAGS.numsamples,2])     
    # Square distance of each point to origin (0,0)
    distances = tf.reduce_sum(tf.pow(i,2), 1) 
    
    # Testing summary: histogram of points' square distantes
    tf.summary.histogram("Testing_histogram", distances)    
    
    return (tf.reduce_sum(tf.cast(tf.greater_equal(tf.cast(1.0,tf.float64), distances),tf.int32))) 


with tf.device("/job:worker/task:0"): 
    test1 = generate_sum() 

with tf.device("/job:worker/task:1"): 
    test2 = generate_sum() 
    

with tf.Session("grpc://" + test_tf_cluster["worker"][0]) as sess: 
       
    test = tf.constant(1)    
    tf.summary.scalar("Testing_constant", test)    
    
    merged = tf.summary.merge_all()
    
    summary, result = sess.run([ merged,
                                 tf.cast(test1 + test2, tf.float64)/FLAGS.numsamples*2.0  ]) 
    
    # Create a summary writer, add the 'graph' to the event file.
    writer = tf.summary.FileWriter("/tmp/tensorboard/test2", sess.graph)   
    writer.add_summary(summary, 1)
    writer.flush()
    
# Reset the default graph
tf.reset_default_graph()
    